In [1]:
import tensorflow as tf
import numpy as np


n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10


In [2]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [3]:
model = tf.keras.Sequential((
    tf.keras.layers.Dense(units=n_hidden1, activation=tf.nn.elu),
    tf.keras.layers.BatchNormalization(momentum=0.9),
    # tf.nn.elu(),
    tf.keras.layers.Dense(units=n_hidden2, activation=tf.nn.elu),
    tf.keras.layers.BatchNormalization(momentum=.9, trainable=True),
    # tf.nn.elu(),
    tf.keras.layers.Dense(units=n_outputs, activation=tf.nn.softmax),
    # tf.keras.layers.Softmax(axis=0)
))


In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size

    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

learning_rate = 0.01

def loss_sparse(labels, logits):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

def run_training(X, y):
    with tf.GradientTape() as g:
        pred = model(X)
        loss = loss_sparse(labels=y, logits=pred)

    gradients = g.gradient(loss, model.trainable_variables)
    # print("Gradient ", gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


In [5]:
num_epochs = 20
batch_size = 64

for epoch in range(num_epochs):
    loss = 10
    for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size=batch_size):
        loss = run_training(X_batch, y_batch)
    print('Epoch %d Loss %.4f' % (epoch + 1, loss))

Epoch 1 Loss 1.8687


KeyboardInterrupt: 

In [106]:
model.evaluate(X_valid, y_valid)

5000/5000 [==============================] - 0s 84us/sample - loss: 1.3430 - accuracy: 0.9782


[1.3430144432438305, 0.9782]

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=20)

In [ ]:
model.evaluate(X_valid, y_valid)